# TF-IDF 개념

#### 똑같이 자주 나오는 단어라고 해도, 모든 문서에 고르게 자주 나오는 단어는 특별한 의미가 없는 경우가 많다. 
#### 핸드폰 리뷰에서는 당연히 핸드폰이라는 단어가 자주 쓰이게지만, 어차피 모든 리뷰에 많이 나온다면 리뷰들 사이의 차이를 분석할 때는 크게 도움이 되지 않는다. 
#### TF-IDF는 이러한 측면을 고려하여 단어의 빈도를 보정하는 방법이다.

### TF: 단어(Term)의 등장 빈도(Frequency)
### IDF: 특정 단어가 등장한 문서(Document)의 빈도(Frequency)의 역수(Inverse)
#### -> 총문서의 수를 각 단어가 등장한 문서의 수(=df)로 나눈 값

# 데이터 불러오기

In [1]:
import pandas as pd
df = pd.read_excel('../data/imdb.xlsx', index_col=0)
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


# TF-IDF

### scikit-learn 패키지의 TfidfVectorizer 를 사용한다.

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

### TfidfVectorizer의 사용법은 CountVectorizer와 거의 동일하다.

#### stop_words : 분석에서 제외할 불용어를 설정하는 옵션이다. stop_words=english로 설정하면 영어의 경우 관사, 전치사 등을 제외한다.
#### max_features : 단어문서행렬에 포함시킬 최대(max)의 단어(feature) 수를 말한다. 즉, 빈도 순으로 최대 500 단어까지 포함한다.

In [3]:
tfidf = TfidfVectorizer(max_features=500, stop_words='english')

### 결과를 tdm으로 저장한다.

In [4]:
tdm = tfidf.fit_transform(df['review'])

### 1000개 문서의 500개 피처가 결과로 반환된다.

In [5]:
tdm

<748x500 sparse matrix of type '<class 'numpy.float64'>'
	with 3434 stored elements in Compressed Sparse Row format>

# 단어 빈도순 정렬

### word_count 변수에 단어와 tf-idf를 저장한다.

In [6]:
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    'tf-idf': tdm.sum(axis=0).flat
})

In [7]:
word_count

,단어,tf-idf
0,10,11.338887
1,20,0.994978
2,90,2.259978
3,absolutely,3.531848
4,acted,1.263767
...,...,...
495,wrong,1.191511
496,year,1.407039
497,years,3.622302
498,yes,1.390106


### TF-IDF의 내림차순으로 정렬하여 상위 5단어를 확인한다.

In [8]:
word_count.sort_values('tf-idf', ascending=False).head(5)

,단어,tf-idf
284,movie,44.917213
153,film,40.356390
33,bad,25.258572
225,just,20.296871
178,good,18.604656


# TfidfTransformer

### CountVectorizer로 만든 단어 문서 행렬을 변환하는 방법으로 TF-IDF를 적용할 수도 있다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

### 먼저 CountVectorizer로 단어 문서 행렬을 만든다.

In [10]:
cv = CountVectorizer(max_features=500, stop_words='english')
tdm2 = cv.fit_transform(df['review'])

### 다음으로 TF-IDF를 적용한다.

In [11]:
trans = TfidfTransformer()
tdm3 = trans.fit_transform(tdm2)

# 두 가지 방법으로 만든 TDM의 비교

In [12]:
import numpy as np
# close 가깝다 -> allclose(): 모든 값이 가까운지를 확인해주는 함수
# tdm.A -> 압축해제
np.allclose(tdm.A, tdm3.A) 

True

# 정렬해서 보기

In [13]:
wc2 = pd.DataFrame({
    '단어': cv.get_feature_names(),
    'tf-idf': tdm3.sum(axis=0).flat
})

wc2.sort_values('tf-idf', ascending=False).head()

,단어,tf-idf
284,movie,44.917213
153,film,40.356390
33,bad,25.258572
225,just,20.296871
178,good,18.604656
